In [45]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

class TradingStrategy:
    """
    Base class for trading strategy simulations.

    Attributes:
        dataframe (pandas.DataFrame): The trading data including signals.
        initial_balance (float): The initial balance for trading.
        signal_length (int): The length of the trading signals.
        shares (list): List to track shares held at each time step.
        capital (list): List to track capital at each time step.
        balance (list): List to track balance at each time step.
        rate (float): Transaction fee rate to consider into the trade.
    """
    def __init__(self, dataframe, initial_balance):
        self.dataframe = dataframe
        self.initial_balance = initial_balance
        self.signal_length = dataframe.shape[0]
        self.shares = [0] * self.signal_length
        self.capital = [0] * self.signal_length
        self.balance = [0] * self.signal_length
        self.rate = 0.9975

    def simulate(self):
        """
        Simulate the trading strategy.

        This method should be implemented by subclasses.
        """
        raise NotImplementedError("Subclasses must implement the 'simulate' method.")

    def calculate_returns(self):
        """
        Calculate and add returns to the dataframe.

        This method calculates the returns based on the capital and initial balance.
        """
        self.dataframe['Return'] = self.dataframe['Capital'].pct_change(1)
        self.dataframe.dropna(inplace=True)

class BuyAndHold(TradingStrategy):
    """
    Simulate a Buy and Hold trading strategy.

    This class simulates the Buy and Hold trading strategy using the provided trading data and initial balance.

    Attributes:
        Inherits attributes from TradingStrategy.
    """
    def simulate(self):
        first_buy = self.dataframe.loc[self.dataframe['Signal'] == 1].head(1).index.values[0]
        last_sell = self.dataframe.loc[self.dataframe['Signal'] == 0].tail(1).index.values[0]

        for i in range(first_buy):
            self.capital[i] = self.initial_balance
            self.balance[i] = self.initial_balance
            self.shares[i] = 0

        self.shares[first_buy] = self.initial_balance // self.dataframe['Open'][first_buy]
        self.balance[first_buy] = self.initial_balance % self.dataframe['Open'][first_buy]
        self.capital[first_buy] = self.rate * (self.shares[first_buy] * self.dataframe['Open'][first_buy]) + self.balance[first_buy]

        for i in range(first_buy + 1, last_sell + 1):
            self.shares[i] = self.shares[i - 1]
            self.balance[i] = self.balance[i - 1]
            self.capital[i] = self.balance[i - 1] + self.shares[i] * self.dataframe['Close'][i]

        self.capital[last_sell + 1] = self.balance[last_sell] + self.rate * (self.shares[last_sell] * self.dataframe['Open'][last_sell + 1])
        self.balance[last_sell + 1] = self.capital[last_sell + 1]
        self.shares[last_sell + 1] = 0

        for i in range(last_sell + 2, self.signal_length):
            self.shares[i] = 0
            self.balance[i] = self.balance[i - 1]
            self.capital[i] = self.capital[i - 1]

        self.dataframe = self.dataframe.assign(Shares=pd.Series(self.shares), Capital=pd.Series(self.capital), Balance=pd.Series(self.balance))

class BuyAndSell(TradingStrategy):
    """
    Simulate a Buy and Sell trading strategy.

    This class simulates the Buy and Sell trading strategy using the provided trading data and initial balance.

    Attributes:
        Inherits attributes from TradingStrategy.
    """
    def simulate(self):
        first_buy = self.dataframe.loc[self.dataframe['Signal'] == 1].head(1).index.values[0]
        signal_length = self.dataframe.shape[0]

        shares = [0] * signal_length
        balance = [0] * signal_length
        capital = [0] * signal_length

        for i in range(first_buy):
            capital[i] = self.initial_balance
            balance[i] = self.initial_balance

        for i in range(first_buy, signal_length):
            if self.dataframe['Signal'][i] == 1 and self.dataframe['Signal'][i - 1] == 0:
                shares[i] = balance[i - 1] // self.dataframe['Close'][i]
                balance[i] = balance[i - 1] % self.dataframe['Close'][i]
                capital[i] = self.rate * (shares[i] * self.dataframe['Close'][i]) + balance[i]
            elif self.dataframe['Signal'][i] == 1 and self.dataframe['Signal'][i - 1] == 1:
                shares[i] = shares[i - 1]
                capital[i] = capital[i - 1]
                balance[i] = balance[i - 1]
            elif self.dataframe['Signal'][i] == 0 and self.dataframe['Signal'][i - 1] == 1:
                balance[i] = balance[i - 1] + self.rate  * (shares[i - 1] * self.dataframe['Close'][i])
                capital[i] = balance[i]
                shares[i] = 0
            elif self.dataframe['Signal'][i] == 0 and self.dataframe['Signal'][i - 1] == 0:
                shares[i] = 0
                capital[i] = capital[i - 1]
                balance[i] = balance[i - 1]

        self.dataframe = self.dataframe.assign(Shares=pd.Series(shares), Capital=pd.Series(capital), Balance=pd.Series(balance))

class Statistics:
    """
    Calculate and visualize trading statistics.

    This class calculates and visualizes various trading statistics including Sharpe ratio and maximum drawdown.

    Attributes:
        returns (list): List of returns.
        dates (list): List of dates corresponding to the returns.
        capitals (list): List of capital values.
        initial_balance (float): The initial balance for trading.
        market (str): Market identifier.
        stock (str): Stock identifier.
        method (str): Trading method identifier.
        trade (bool): True for trading strategy, False for buy and hold.
        ror (list): List to store the rate of return.
        drawdowns (list): List to store drawdowns.
    """
    def __init__(self, returns, dates, capitals, initial_balance, market, stock, method, trade=True):
        self.returns = returns
        self.dates = dates
        self.capitals = capitals
        self.initial_balance = initial_balance
        self.market = market
        self.stock = stock
        self.method = method
        self.trade = trade
        self.ror = None
        self.drawdowns = None

    def mean(self):
        """
        Calculate the mean of the returns.

        Returns:
            float: Mean of the returns.
        """
        return np.mean(self.returns)

    def std(self):
        """
        Calculate the standard deviation of the returns.

        Returns:
            float: Standard deviation of the returns.
        """
        return np.std(self.returns)

    def dd(self):
        """
        Calculate and store drawdowns based on the rate of return.

        This method computes and stores the drawdowns based on the rate of return.
        """
        max_ror = max(self.ror)
        rm = np.maximum.accumulate(self.ror)
        rm[rm < 1] = 1
        self.drawdowns = self.ror / rm - 1

    def mdd(self):
        """
        Calculate and return the maximum drawdown.

        Returns:
            float: Maximum drawdown.
        """
        money = np.cumprod(1 + self.returns)
        maximums = np.maximum.accumulate(money)
        self.drawdowns = (1 - money / maximums) * 100
        return np.max(self.drawdowns)

    def sharpe(self, risk_free_rate=0.0003):
        """
        Calculate the Sharpe ratio.

        Args:
            risk_free_rate (float, optional): The risk-free rate. Default is 0.0003.

        Returns:
            float: Sharpe ratio.
        """
        mu = self.mean()
        sig = self.std()
        sharpe_d = (mu - risk_free_rate) / sig
        return (252 ** 0.5) * sharpe_d

    def compute_ror(self):
        """
        Calculate the rate of return (RoR).

        This method calculates the rate of return based on the capital and initial balance.
        """
        self.ror = ((self.capitals - self.initial_balance) / self.initial_balance) * 100

    def plot_capital(self):
        """
        Plot the capital over time.
        """
        fig, ax = plt.subplots()
        plt.plot(self.dates, self.capitals)
        plt.title(self.title)
        plt.ylabel('Capital Amount')
        ax.xaxis.set_tick_params(rotation=30, labelsize=10)
        plt.show()

    def markowitz(self):
        """
        Calculate Markowitz portfolio weights.

        Returns:
            numpy.ndarray: Portfolio weights.
        """
        numinator = np.transpose(self.returns).dot(self.returns)
        denuminator = self.returns.shape[0]
        sigma = numinator / denuminator
        mu = self.mean()
        soorat = np.linalg.pinv(sigma)
        w = np.dot(soorat, mu)
        return w

    def plot_dd(self):
        """
        Plot the drawdowns over time.
        """
        fig, ax = plt.subplots()
        plt.plot(self.dates, self.drawdowns)
        plt.title(self.title)
        plt.ylabel('Drawdowns %')
        ax.xaxis.set_tick_params(rotation=30, labelsize=10)
        plt.show()

    def plot_ror(self):
        """
        Plot the rate of return (RoR) over time.
        """
        fig, ax = plt.subplots()
        self.compute_ror()
        plt.plot(self.dates, self.ror)
        plt.title(self.title)
        plt.ylabel('RoR%')
        ax.xaxis.set_tick_params(rotation=30, labelsize=10)
        plt.show()

    def get_metrics(self):
        """
        Calculate and return Sharpe ratio and maximum drawdown.

        Returns:
            tuple: Sharpe ratio and maximum drawdown.
        """
        return self.sharpe(), self.mdd()

    def report(self):
        """
        Print and visualize trading statistics.

        This method prints the Sharpe ratio and maximum drawdown and creates plots for drawdowns, rate of return, and capital.
        """
        print('Sharpe ratio \t', self.sharpe())
        print('Maximum Drawdown \t', self.mdd())
        self.plot_dd()
        self.plot_ror()
        self.plot_capital()

        
def run_strategy(strategy, names, Market, Method, HHT):

    """
    Run trading strategies for a list of stock names.

    Args:
        names (list): List of stock names.
        Market (str): Market identifier.
        Method (str): Trading method identifier.
        HHT (bool): True if using HHT, False for raw data.

    Returns:
        dict: A dictionary of trading metrics 'Sharpe ratio and MDD' for each stock.
    """
    
    dict_metrics = {}
    print(names)
    for stock in names:
        if HHT:
            filename = 'Data/HHT/Signal/' + Market + '/' + stock + '_signal_' + Method + '.csv'
            filename_prefix = 'Result/' + Market + '/HHT/Trade/' + stock
        else:
            filename = 'Data/Raw/Signal/' + Market + '/' + stock + '_signal_' + Method + '.csv'
            filename_prefix = 'Result/' + Market + '/Trade/' + stock

        df = pd.read_csv(filename, index_col=None, parse_dates=['Date'])
        trading_strategy = strategy(df.copy(), InitialBalance)
        trading_strategy.simulate()
        trading_strategy.calculate_returns()

        statistics = Statistics(trading_strategy.dataframe['Return'], trading_strategy.dataframe['Date'], trading_strategy.dataframe['Capital'], InitialBalance,
                                Market, stock, Method, trade=True)
        sr, mdd = statistics.get_metrics()
        dict_metrics[stock] = {'SR': np.round(sr, 5), 'MDD': np.round(mdd, 5)}

    return dict_metrics
if __name__ == "__main__":
    Market = 'SP500'
    Method = 'XGB'
    InitialBalance = 1e+6
    path = 'Data/Raw/' + Market + '/*.csv'
    all_files = glob.glob(path)
    # names maintains all stocks in the path
    names = []

    for filename in all_files:
        key = filename.split('\\')[len(filename.split('\\')) - 1].replace('.csv', '')
        names.append(key)

    # change the starategy to BuyAndHold for simulating the Buy&Hold strategy
    # change the starategy to BuyAndSell for simulation the Buy&Sell strategy
    strategy = BuyAndSell

    dict_metrics = run_strategy(strategy, names, Market, Method, HHT=True)        

['AAP', 'AAPL', 'ABT', 'ACN', 'ADBE', 'AES', 'AFL', 'AIV', 'AJG', 'AKAM', 'AMD', 'AMZN', 'APH', 'ARE', 'ATVI', 'AZO', 'BA', 'CAH', 'CB', 'CBOE', 'CCI', 'CLX', 'CMS', 'COF', 'COO', 'CPB', 'CSX', 'CVS', 'DIS', 'DVN', 'DXC', 'EBAY', 'ETR', 'FIS', 'FISV', 'GD', 'GILD', 'GOOG', 'GPC', 'GT', 'GWW', 'HAS', 'HBI', 'HSIC', 'HST', 'IBM', 'INCY', 'INTC', 'INTU', 'IP', 'IT', 'JPM', 'JWN', 'K', 'KIM', 'KMX', 'KSS', 'LYB', 'MAR', 'MCO', 'MDLZ', 'MDT', 'MMM', 'MSFT', 'MU', 'NEE', 'NEM', 'PDCO', 'PH', 'PRU', 'QCOM', 'RHI', 'RSG', 'SBUX', 'SLB', 'SLG', 'SNA', 'SPG', 'SRE', 'SYY', 'TSCO', 'TSLA', 'UAA', 'UAL', 'UNP', 'UPS', 'URI', 'V', 'VLO', 'VMC', 'VNO', 'WFC', 'WMB', 'WU', 'WYNN']


In [46]:
dict_metrics

{'AAP': {'SR': 5.05383, 'MDD': 3.85102},
 'AAPL': {'SR': 4.6811, 'MDD': 2.95118},
 'ABT': {'SR': 4.51277, 'MDD': 2.01791},
 'ACN': {'SR': 4.51932, 'MDD': 4.41465},
 'ADBE': {'SR': 5.0195, 'MDD': 2.35759},
 'AES': {'SR': 4.95584, 'MDD': 5.43748},
 'AFL': {'SR': 4.33501, 'MDD': 1.96935},
 'AIV': {'SR': 4.75954, 'MDD': 4.0575},
 'AJG': {'SR': 4.55579, 'MDD': 1.82503},
 'AKAM': {'SR': 4.04587, 'MDD': 4.68169},
 'AMD': {'SR': 5.25439, 'MDD': 3.0207},
 'AMZN': {'SR': 3.97112, 'MDD': 2.6346},
 'APH': {'SR': 4.26526, 'MDD': 4.24037},
 'ARE': {'SR': 4.06012, 'MDD': 4.19696},
 'ATVI': {'SR': 3.47161, 'MDD': 2.83436},
 'AZO': {'SR': 4.77998, 'MDD': 5.71373},
 'BA': {'SR': 3.97682, 'MDD': 4.55198},
 'CAH': {'SR': 4.30603, 'MDD': 3.04612},
 'CB': {'SR': 4.97601, 'MDD': 2.01926},
 'CBOE': {'SR': 5.0024, 'MDD': 1.54638},
 'CCI': {'SR': 4.18053, 'MDD': 3.33771},
 'CLX': {'SR': 2.98427, 'MDD': 7.01791},
 'CMS': {'SR': 4.47216, 'MDD': 2.11306},
 'COF': {'SR': 5.1163, 'MDD': 4.397},
 'COO': {'SR': 4.5369